In [5]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [1]:
import mlflow

from src.libs.configs import MLFLOW_FAKE_NEWS_EXPERIMENT_NAME, MLFLOW_TRACKING_URI

In [2]:

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("fake_news_poc_experiment")

experiments= mlflow.search_experiments()
for exp in experiments:
    print(exp.name)

https://dagshub.com/abdala9512/fake-news-poc.mlflow/#/experiments/0
fake_news_poc_experiment


In [4]:
all_experiments = mlflow.search_runs()
all_experiments

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_auc_15,metrics.val_loss,metrics.auc_15,metrics.loss,...,params.var_smoothing,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.git.commit,tags.mlflow.user,tags.estimator_class,tags.estimator_name,tags.dagshub.labels.hidden
0,8d90c4136cb04f95b7d905addacd168c,0,FINISHED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-05-22 17:28:55.224000+00:00,2023-05-22 17:29:43.161000+00:00,0.808723,2.170157,1.0,0.000201,...,None,"[{""run_id"": ""8d90c4136cb04f95b7d905addacd168c""...",src/pipeline_training_full.py,LOCAL,respected-fox-149,27e7132a70b588529b855a1ca7a8ff24787d134f,abdala9512,None,None,None
1,afc5a0da876e47ea84cf83cd3064ed25,0,FINISHED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-05-22 04:13:22.793000+00:00,2023-05-22 04:14:21.944000+00:00,0.878798,1.318613,1.0,0.004409,...,None,"[{""run_id"": ""afc5a0da876e47ea84cf83cd3064ed25""...",src/pipeline_training_full.py,LOCAL,powerful-kit-154,27e7132a70b588529b855a1ca7a8ff24787d134f,abdala9512,None,None,None
2,d9913fd44ed64473acb7045b6ea428ef,0,FINISHED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-05-22 04:02:16.550000+00:00,2023-05-22 04:03:16.720000+00:00,NaN,1.965782,NaN,0.003671,...,None,"[{""run_id"": ""d9913fd44ed64473acb7045b6ea428ef""...",src/pipeline_training_full.py,LOCAL,bemused-shrimp-839,81d5b8603a92f7cc7cdd25e4e75857b7a6914f24,abdala9512,None,None,None
3,620eb542cf8c4080b008c5cbad068ee5,0,FAILED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-05-22 03:50:15.505000+00:00,2023-05-22 03:50:16.360000+00:00,NaN,NaN,NaN,NaN,...,None,None,src/pipeline_training_full.py,LOCAL,judicious-rat-605,81d5b8603a92f7cc7cdd25e4e75857b7a6914f24,abdala9512,None,None,None
4,93c492c817b646728cae5d1f4de1176a,0,FAILED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-05-22 03:41:50.330000+00:00,2023-05-22 03:41:51.672000+00:00,NaN,NaN,NaN,NaN,...,None,None,src/pipeline_training_full.py,LOCAL,polite-horse-427,81d5b8603a92f7cc7cdd25e4e75857b7a6914f24,abdala9512,None,None,None
5,d7cd3ad3018e4c1182fc335135414102,0,FAILED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-05-22 03:39:53.195000+00:00,2023-05-22 03:39:54.534000+00:00,NaN,NaN,NaN,NaN,...,None,None,src/pipeline_training_full.py,LOCAL,placid-deer-389,81d5b8603a92f7cc7cdd25e4e75857b7a6914f24,abdala9512,None,None,None
6,8593eac1e56942feaf3bc360f83e95d7,0,FINISHED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-04-02 22:01:44.416000+00:00,2023-04-02 22:01:54.241000+00:00,NaN,NaN,NaN,NaN,...,1e-09,"[{""run_id"": ""8593eac1e56942feaf3bc360f83e95d7""...",/home/abdala9512/.local/share/virtualenvs/fake...,LOCAL,kindly-sow-9,None,abdala9512,sklearn.naive_bayes.GaussianNB,GaussianNB,None
7,b3470133af0e46bdadff5dc5597ed889,0,FAILED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-04-02 22:00:20.263000+00:00,2023-04-02 22:00:28.929000+00:00,NaN,NaN,NaN,NaN,...,1e-09,"[{""run_id"": ""b3470133af0e46bdadff5dc5597ed889""...",/home/abdala9512/.local/share/virtualenvs/fake...,LOCAL,learned-bear-576,None,abdala9512,sklearn.naive_bayes.GaussianNB,GaussianNB,-
8,4425700ba54b4f3c951940931c6abd17,0,FAILED,mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d6...,2023-04-02 21:59:59.716000+00:00,2023-04-02 22:00:11.872000+00:00,NaN,NaN,NaN,NaN,...,1e-09,"[{""run_id"": ""4425700ba54b4f3c951940931c6abd17""...",/home/abdala9512/.local/share/virtualenvs/fake...,LOCAL,whimsical-sheep-855,None,abdala9512,sklearn.naive_bayes.GaussianNB,GaussianNB,-


In [5]:
valid_runs = all_experiments[~all_experiments["metrics.val_auc_15"].isna()]

In [6]:
best_run = valid_runs.loc[valid_runs['metrics.val_auc_15'].idxmax()]["run_id"]

In [7]:
best_run

'afc5a0da876e47ea84cf83cd3064ed25'

In [28]:
from typing import List
import json
from src.libs.configs import MLFLOW_FAKE_NEWS_EXPERIMENT_NAME, MLFLOW_TRACKING_URI, MLFLOW_FAKE_NEWS_MODEL_NAME
from dotenv import load_dotenv
import mlflow

load_dotenv()  # take environment variables from .env.


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_FAKE_NEWS_EXPERIMENT_NAME)

def search_best_model(
        experiment_names: List[str] = [MLFLOW_FAKE_NEWS_EXPERIMENT_NAME],
        metric_name: str = "val_auc_1"
    ) -> str:
    """Search Best Run ID of given experiments
    """
    runs_  = mlflow.search_runs(experiment_names=experiment_names)
    run_id = runs_.loc[runs_[f'metrics.{metric_name}'].idxmax()]['run_id']
    artifact_path = json.loads(
        runs_[runs_["run_id"] == run_id]["tags.mlflow.log-model.history"].values[0]
    )[0]["artifact_path"]
    
    return run_id, artifact_path

best_ml_model_id, artifact_path = search_best_model()
best_ml_model_id

'21611200625f46c2a22f761741d2e4d0'

In [29]:
result = mlflow.register_model(
    f"runs:/{best_ml_model_id}/{artifact_path}",
    MLFLOW_FAKE_NEWS_MODEL_NAME
)

Registered model 'fake_news_poc_model' already exists. Creating a new version of this model...
2023/05/22 13:38:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: fake_news_poc_model, version 5
Created version '5' of model 'fake_news_poc_model'.


In [31]:

from mlflow import MlflowClient

def get_artifact_uri_production() -> str:

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{MLFLOW_FAKE_NEWS_MODEL_NAME}'"):
        model = dict(mv)
        if model["current_stage"] == "Production":
            production_model = model

    _run_id = production_model.get("run_id")
    return mlflow.get_run(_run_id).info.artifact_uri


'mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d60a9b/21611200625f46c2a22f761741d2e4d0/artifacts'

In [33]:
BEST_PARAMS_PROD = f"{get_artifact_uri_production()}/best_params.json"

mlflow.artifacts.load_dict(BEST_PARAMS_PROD)

{'BATCH_SIZE': 128,
 'DENSE_SIZE': 32,
 'DROPOUT': 0.2879062020350355,
 'EMBED_DIM': 128,
 'EPOCHS': 20,
 'LSTM_SIZE': 128,
 'MAXLEN': 300,
 'OPTIMIZER': 'rmsprop',
 'VOCAB_SIZE': 22129}

In [25]:
mlflow.get_run("afc5a0da876e47ea84cf83cd3064ed25").info.artifact_uri

'mlflow-artifacts:/052bdf9ed0ab421080a4d421f8d60a9b/afc5a0da876e47ea84cf83cd3064ed25/artifacts'

In [2]:
stage = "Production"
mlflow.tensorflow.load_model(model_uri=f"models:/{MLFLOW_FAKE_NEWS_MODEL_NAME}/Production",)

2023-05-22 13:02:52.830179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 13:02:52.995981: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-22 13:02:52.998343: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-22 13:02:52.998353: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [19]:
! pip install tensorflow==2.11.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 7.1 MB/s eta 0:00:007.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 12.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
  Attempting uninstall: tensorflow
    Found existing installat